In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
#====================================================================
# calling data
trans = transforms.Compose([transforms.Resize((200,200)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
trainset = torchvision.datasets.ImageFolder(root="/home/gus/catvsdog",
                                           transform = trans)
train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size = 32,
                                          shuffle=True)
#====================================================================
# declaring hyperparameters
lr = 0.01
#====================================================================
# constructiong CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # input shape = (?, 200, 200, 3)
        # Conv -> (?, 200, 200, 16)
        # Pool -> (?, 100, 100, 16)
        self.layer = nn.Sequential(
            torch.nn.Conv2d(3,16,3,stride=1,padding=1),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2),
        # input shape = (?, 100, 100, 16)
        # Conv -> (?, 100, 100, 32)
        # Pool -> (?, 50, 50, 32)
            torch.nn.Conv2d(16,32,3,stride=1,padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = torch.nn.Linear(50*50*32,2,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self,x):
        out = self.layer(x)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
#====================================================================
# class EarlyStopping():
#     def __init__(self, patience=0, verbose=0):
#         self._step = 0
#         self._loss = float('inf')
#         self.patience  = patience
#         self.verbose = verbose

#     def validate(self, loss):
#         if self._loss < loss:
#             self._step += 1
#             if self._step > self.patience:
#                 if self.verbose:
#                     print(f'Training process is stopped early....')
#                 return True
#         else:
#             self._step = 0
#             self._loss = loss

#         return False
#====================================================================
model=CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.RMSprop(model.parameters(),lr = lr)

total_batch = len(train_loader)

for epoch in range(20):
    avg_cost = 0
    for X,Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
#     ES = EarlyStopping(patience=10,verbose=1)
#     if ES(cost):
#         break

[Epoch:    1] cost = 70.9169388
[Epoch:    2] cost = 0.973889232
[Epoch:    3] cost = 0.622291446
[Epoch:    4] cost = 0.604466677
[Epoch:    5] cost = 0.729211032
[Epoch:    6] cost = 0.557634294
[Epoch:    7] cost = 0.483414382
[Epoch:    8] cost = 0.648886025
[Epoch:    9] cost = 0.368263602
[Epoch:   10] cost = 0.294967532
[Epoch:   11] cost = 0.230564445
[Epoch:   12] cost = 0.205529153
[Epoch:   13] cost = 0.149951935
[Epoch:   14] cost = 0.142383158
[Epoch:   15] cost = 0.616265297
[Epoch:   16] cost = 0.0851518139
[Epoch:   17] cost = 0.0658707321
[Epoch:   18] cost = 0.0951854214
[Epoch:   19] cost = 0.0875103399
[Epoch:   20] cost = 0.133087859


In [5]:
# calling data
trans = transforms.Compose([transforms.Resize((200,200)),
                           transforms.ToTensor(),
                           transforms.Normalize((-1,-1,-1),(2,2,2))])
testset = torchvision.datasets.ImageFolder(root="/home/gus/cattest",
                                           transform = trans)
test_loader = torch.utils.data.DataLoader(testset,
                                          batch_size = 32,
                                          shuffle=True)
test_epoch = 0
with torch.no_grad():
    for X,Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        correct_prediction = torch.argmax(prediction,1) == Y
        accuracy = correct_prediction.float().mean()
        test_epoch += 1
        print('Accuracy:',accuracy.item())
        if test_epoch == 10:
            break


Accuracy: 0.71875
Accuracy: 0.6875
Accuracy: 0.5625
Accuracy: 0.65625
Accuracy: 0.5625
Accuracy: 0.75
Accuracy: 0.6875
Accuracy: 0.75
Accuracy: 0.6875
Accuracy: 0.65625
